In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
corpus_path = "../../corpus/SNLI/"
data_name = "dev_sentence.csv"

In [3]:
df = pd.read_csv(corpus_path+data_name)

In [4]:
df

,label,pre,hypo
0,contradiction,家の前の雪の山の前に立っている数人の人と、シャベルを使っている人。,マイアミの暑い夏の日です。
1,contradiction,人が２頭の馬の間にひざまずいている,２頭の牛の間に人が立っています。
2,contradiction,人は森の中の崖に登り、他の人は見ています。,人が森の中の木に登る。
3,contradiction,男と女が丘の上を歩きます。,男と女が座っています。
4,contradiction,３人、１人は床に座ってギターを弾き、２人はソファでゲームをします。,数人がビーチに座っています。
...,...,...,...
3911,neutral,バセットハウンドは、男と女の前の路地の戸口に縛られています。,それは彼らの犬です。
3912,neutral,ローラーダービーの前にストレッチ。,競争の前にストレッチをしている人もいます。
3913,neutral,アクロバティックな動きを実行するステージ上の２人の芸能人。,２人の芸能人が異常に大勢の観客のためにアクロバットを行っています。
3914,neutral,バスケットボールコートの２人の男性が１人は白で、もう１人は青で、白の選手はチームメイトが見な...,男性はバスケットボールトーナメントでプレーしています。


In [5]:
df['label'].value_counts()

entailment       1432
neutral          1328
contradiction    1156
Name: label, dtype: int64

In [6]:
df.dropna(how="all")

,label,pre,hypo
0,contradiction,家の前の雪の山の前に立っている数人の人と、シャベルを使っている人。,マイアミの暑い夏の日です。
1,contradiction,人が２頭の馬の間にひざまずいている,２頭の牛の間に人が立っています。
2,contradiction,人は森の中の崖に登り、他の人は見ています。,人が森の中の木に登る。
3,contradiction,男と女が丘の上を歩きます。,男と女が座っています。
4,contradiction,３人、１人は床に座ってギターを弾き、２人はソファでゲームをします。,数人がビーチに座っています。
...,...,...,...
3911,neutral,バセットハウンドは、男と女の前の路地の戸口に縛られています。,それは彼らの犬です。
3912,neutral,ローラーダービーの前にストレッチ。,競争の前にストレッチをしている人もいます。
3913,neutral,アクロバティックな動きを実行するステージ上の２人の芸能人。,２人の芸能人が異常に大勢の観客のためにアクロバットを行っています。
3914,neutral,バスケットボールコートの２人の男性が１人は白で、もう１人は青で、白の選手はチームメイトが見な...,男性はバスケットボールトーナメントでプレーしています。


In [7]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models ,losses
from sentence_transformers.readers import InputExample
from sentence_transformers.losses import TripletDistanceMetric, SoftmaxLoss
from sentence_transformers.evaluation import TripletEvaluator
from sentence_transformers.readers import TripletReader
from sentence_transformers.datasets import SentencesDataset
from torch.utils.data import DataLoader

In [8]:
label = ["entailment", "neutral", "contradiction"]
label2id = dict( zip(label, range(len(label))) )

def make_dataset_snli(df, max_n = 300):
    X = []
    each_label_num = [0, 0 ,0]
    for la, pre, hypo in zip(df.label, df.pre, df.hypo):
        if each_label_num[label2id[la]] >= max_n:
            continue
        X.append( InputExample(texts=[pre, hypo], label=label2id[la] ) )
        each_label_num[label2id[la]] += 1
    return X


In [18]:
X = make_dataset_snli(df, max_n=5000)

In [19]:
X_train, X_test = train_test_split(X, train_size=0.8, random_state=4)

In [13]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models

download_path = "../../corpus/pretrained/"
# download_path = ""
transformer = models.Transformer(download_path+'cl-tohoku/bert-base-japanese-whole-word-masking')
pooling = models.Pooling(transformer.get_word_embedding_dimension(),    
  pooling_mode_mean_tokens=True,
  pooling_mode_cls_token=False, 
  pooling_mode_max_tokens=False
)
model = SentenceTransformer(modules=[transformer, pooling])

sentences = ['吾輩は猫である',  '本日は晴天なり']
embeddings = model.encode(sentences)

for i, embedding in enumerate(embeddings):
  print("[%d] : %s" % (i, embedding.shape, ))

Some weights of the model checkpoint at ../../corpus/pretrained/cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[0] : (768,)
[1] : (768,)


In [20]:
len(X)

3916

In [24]:
BATCH_SIZE = 32
NUM_EPOCHS = 3
EVAL_STEPS = 1000
WARMUP_STEPS = int(len(X_train) // BATCH_SIZE * 0.1) 
OUTPUT_PATH = "../../corpus/sbert_snli"

In [25]:
train_data = SentencesDataset(X_train, model=model)
train_dataloader = DataLoader(train_data, shuffle=True,  batch_size=BATCH_SIZE)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=len(label2id))

In [26]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
         epochs=NUM_EPOCHS,
         evaluation_steps=EVAL_STEPS,
         warmup_steps=WARMUP_STEPS,
         output_path=OUTPUT_PATH
         )

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/98 [00:00<?, ?it/s]

Iteration:   0%|          | 0/98 [00:00<?, ?it/s]

Iteration:   0%|          | 0/98 [00:00<?, ?it/s]

In [27]:
import torch

torch.cuda.is_available()

True